#### Advanced Statistics for Data Science (Spring 2022)
# Home Assignment 4
#### Topics:
- ANOVA


#### Due: 10/05/2022 by 18:30

#### Instructions:
- Write your name, Student ID, and date in the cell below. 
- Submit a copy of this notebook with code filled in the relevant places as the solution of coding excercises.
- For theoretic excercises, you can either write your solution in the notebook using $\LaTeX$ or submit additional notes.

<hr>
<hr>


**Name**: Eyal Michaeli

**Student ID**: 207380528

**Date**: 10.05.22

$
\newcommand{\Id}{{\mathbf{I}}}  
\newcommand{\SSE}{\mathsf{SSE}}
\newcommand{\SSR}{\mathsf{SSR}}
\newcommand{\MSE}{\mathsf{MSE}}
\newcommand{\simiid}{\overset{iid}{\sim}}
\newcommand{\ex}{\mathbb E}
\newcommand{\var}{\mathrm{Var}}
\newcommand{\Cov}[2]{{\mathrm{Cov}  \left(#1, #2 \right)}}
\newcommand{\one}[1]{\mathbf 1 {\left\{#1\right\}}}
\newcommand{\SE}[1]{\mathrm{SE} \left[#1\right]}
\newcommand{\reals}{\mathbb R}
\newcommand{\Ncal}{\mathcal N}
\newcommand{\abs}[1]{\ensuremath{\left\vert#1\right\vert}}
\newcommand{\rank}{\operatorname{rank}}
\newcommand{\tr}{\operatorname{Tr}}
\newcommand{\diag}{\operatorname{diag}}
\newcommand{\sign}{\operatorname{sign}}
$


<hr>
<hr>

# Problem 1 (Solving LS using SVD)
Consider the housing prices dataset (``housing_prices.csv``). Use houses of lot size smaller than 15000 ft.

1. Find the least squares coefficient of the linear model with target variable ``SalePrice`` and the 16 predictors:
``['LotArea',  'YearBuilt',
  'GarageCars', 'YrSold', 'MoSold', 'Fireplaces',
  'HalfBath', 'LowQualFinSF', 'TotalBsmtSF',
  '1stFlrSF', 'LotFrontage', 'ScreenPorch',
   'WoodDeckSF', 'OverallCond', 'BsmtUnfSF']`` 
plus a constant term. Remove all entries in which one or more of these predictors is missing. 

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [4]:

target = 'SalePrice' 
lo_predictors = ['const', 'SalePrice', 'LotArea',  'YearBuilt',
    'GarageCars', 'YrSold', 'MoSold', 'Fireplaces',
    'HalfBath', 'LowQualFinSF', 'TotalBsmtSF',
    '1stFlrSF', 'LotFrontage', 'ScreenPorch',
     'WoodDeckSF', 'OverallCond', 'BsmtUnfSF']

data = pd.read_csv("housing_prices.csv")
data = data[data.LotArea < 15000]  # we focus on small lots
data['const'] = 1                  # add constant term
data = data.filter(lo_predictors).dropna() # remove all other columns

y = data[target].values
X = data.drop(target, axis=1)
Z = X.values
n, p = Z.shape

 - By inverting the matrix $Z^\top Z$. Denote the solution $\hat{\beta}$.
 

In [16]:
A = np.dot(np.linalg.inv(np.dot(Z.T,Z)), Z.T)

betas_hat = np.dot(A, y)

print( ''.join([f"B{i} = {beta_value:.2f}\n" for i, beta_value in enumerate(betas_hat)]))

B0 = -3039719.75
B1 = 2.98
B2 = 616.84
B3 = 21256.55
B4 = 884.95
B5 = 983.79
B6 = 16763.36
B7 = 27915.65
B8 = 68.96
B9 = 43.90
B10 = 41.28
B11 = 0.73
B12 = 22.67
B13 = 30.83
B14 = 7715.94
B15 = -8.10



 - Using the SVD method. Here, decide that $\sigma_i > 0$ if $\sigma_i / \sigma_1 > 10^{-6}$. Denote the solution $\hat{\beta}^{SVD}$.
 

In [13]:
epsilon = 10 ** -6

u, sig, v_t = np.linalg.svd(Z, full_matrices=False)
# np.allclose((u * sig) @ v_t, Z) 
r = np.linalg.matrix_rank(Z, tol=epsilon)
y_star = u.T @ y
beta_star = y_star / sig
beta_hat_svd = v_t.T @ beta_star

print(f"The LS estimator Beta hat is the vector {[round(b, 2) for b in list(beta_hat_svd)]}^T")

The LS estimator Beta hat is the vector [-3039719.746823, 2.978147, 616.839678, 21256.547087, 884.947468, 983.794365, 16763.359425, 27915.651037, 68.960519, 43.896187, 41.27521, 0.733459, 22.668026, 30.830914, 7715.939211, -8.101915]^T


In [18]:
EPSILON = 10 ** -6

u, s, v_t = np.linalg.svd(Z, full_matrices=False)

s_filter = (s / s[0] > EPSILON)
sigma = np.diag(s)

y_star = np.matmul(u.T, y)[:p]

betas_star = np.zeros(p)
betas_star[s_filter] = y_star[s_filter] / s[s_filter]

v = v_t.T
betas_hat_svd = v @ betas_star

print( ''.join([f"B{i} = {beta_value:.2f}\n" for i, beta_value in enumerate(betas_hat_svd)]))

B0 = -0.27
B1 = 2.94
B2 = 612.37
B3 = 21197.77
B4 = -624.39
B5 = 866.65
B6 = 16647.23
B7 = 28030.11
B8 = 68.07
B9 = 44.10
B10 = 41.41
B11 = 6.73
B12 = 23.27
B13 = 31.80
B14 = 7708.39
B15 = -8.18



 - In which method $R^2$ is smaller?
 

2. Plot $\hat{y}$ and $\hat{y}^{SVD}$ over the same pannel to convince yourself that both methods resulted in similar fitted responses. 

3. Plot $\log(|\hat{\beta}_i/\hat{\beta}^{SVD}_i|)$ vs. $i$ for $i=1,\ldots,p$ and indicate the covariate whose coefficient exhibits the largest difference between the methods.

The point: When there are many predictors, it is likely that $Z$ will be rank deficient in the sense that some of its singular values are very small. Removing those singular values is usually a good practice; it is important to observe how this removal affects the solution. 

The largest difference is for the first predictor, which is the constant covariate. We can check that this result makes sense by fiting a new model without the constant covaraite and test whether the residuals in the smaller new model are not significantly different than those in the original model. 

You can use the code below to read and arrange the data

## Problem 2 ($t$-test is a kind of ANOVA)
Consider the following two-sample problem. The data is
$$
y_{ij} = \mu_j + \epsilon_{ij},\qquad j=1,\ldots,n_i,\quad i=0,1. 
$$

1. Find the statistic $t$ for the two-sample $t$ test (using of the observable varaibles)
2. Write the ANOVA table for $k=2$ and find the F statistic $F$ (using of the observable varaibles)
3. Conclude that $t^2 = F$. 
4. Does the test that rejects when $|t| > t_{n-1}^{1-\alpha/2}$ has larger power than the test that rejects when $F > F_{1,n-1}^{1-\alpha}$ ? less? same? 


## Problem 3 (ANOVA Decomposition)
In class, we considered the decomposition:
$$
\begin{equation}
\mathrm{SS}_{tot} = \mathrm{SS}_{within} + \mathrm{SS}_{between}
\label{eq:ANOVA} \tag{1}
\end{equation}
$$
where
$$
\mathrm{SS}_{between} = \sum_{i=1}^k n_i(\bar{y}_{i\cdot} - \bar{y}_{\cdot \cdot})^2,\qquad \mathrm{SS}_{within} = \sum_{i=1}^k \sum_{j=1}^{n_i} (y_{ij} - \bar{y}_{i\cdot})^2
$$
Prove $\eqref{eq:ANOVA}$ by expanding:
$$
\mathrm{SS}_{tot} = \sum_{i=1}^k \sum_{j=1}^{n_i} (y_{ij} - \bar{y}_{\cdot \cdot})^2 = \sum_{i=1}^k \sum_{j=1}^{n_i} (y_{ij} - \bar{y}_{i\cdot} + \bar{y}_{i\cdot} - \bar{y}_{\cdot \cdot})^2 = ...
$$

## Problem 4 (ANOVA and multiple testing in Practice)

Consider Israeli wines in the wine dataset ``winemag-data_first150k.csv`` used in class (downloaded from Kaggle https://www.kaggle.com/datasets/zynicide/wine-reviews?select=winemag-data_first150k.csv). Use ANOVA to measure the effect of winary (``winary``) on the quality (``points``) of wine of veriaty ``Cabernet Sauvignon``. 
1. Load dataset; keep only relevant records according to the fields ``country`` and ``veriaty``; remove winaries with 1 or less samples (because we cannot do ANOVA for those)
2. Plot the boxplot with ``winary`` as the x-axis and ``points`` as the y-axis.
3. Use ANOVA to figure out whether some winaries make better cabernets than others; print the ANOVA table and explain your conclusion
4. Find which winaries have cabernets ranked higher than others:
 - Run all paired t-tests; how many of the test's P-values fall below 0.05? is it more than what is expceted if all tests are null?
 - Use Bonferroni's method to decide which winaries ranked singnificantly higher than others by reporting on the list of pairs whose P-value is significant after Bonferroni's correction. Also indicate which one is ranked higher out of each pair.